In [12]:
#process data

import pandas as pd

x_train = pd.read_parquet("../data/train_data/phase-1/prob-1/train_x.parquet")
y_train = pd.read_parquet("../data/train_data/phase-1/prob-1/train_y.parquet")
x_test = pd.read_parquet("../data/train_data/phase-1/prob-1/test_x.parquet")
y_test = pd.read_parquet("../data/train_data/phase-1/prob-1/test_y.parquet")

n_label_1 = len(y_train.loc[y_train["label"] == 1])
n_label_0 = len(y_train) - n_label_1
scale_pos_weight = n_label_0 / n_label_1
print("scale pos weight:", scale_pos_weight)

scale pos weight: 14.684441197954712


In [17]:
# register model to mlflow
from mlflow.models.signature import infer_signature
import mlflow
# init mlflow
mlflow.set_tracking_uri('http://localhost:5000')
mlflow.set_experiment(
    f"phase-1_prob-1_model-1"
)

<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1687686150727, experiment_id='1', last_update_time=1687686150727, lifecycle_stage='active', name='phase-1_prob-1_model-1', tags={}>

In [13]:
# train model
import xgboost as xgb
model = xgb.XGBClassifier(objective="binary:logistic", scale_pos_weight=scale_pos_weight)
model.fit(x_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [14]:
# evaluate model
from sklearn.metrics import classification_report, roc_auc_score
predictions = model.predict(x_test)
print(classification_report(y_test, predictions))
auc_score = roc_auc_score(y_test, predictions)
metrics = {"test_auc": auc_score}
print(f"metrics: {metrics}")

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     25189
           1       0.94      0.96      0.95      1652

    accuracy                           0.99     26841
   macro avg       0.97      0.98      0.97     26841
weighted avg       0.99      0.99      0.99     26841

metrics: {'test_auc': 0.9791109238370992}


In [18]:
# mlflow log and end experiment
mlflow.log_params(model.get_params())
mlflow.log_metrics(metrics)
signature = infer_signature(x_test, predictions)
mlflow.sklearn.log_model(
    sk_model=model,
    artifact_path='model',
    signature=signature,
)
mlflow.end_run()

/workspaces/mlops-mara-sample-public/.conda/lib/python3.9/site-packages/mlflow/models/signature.py:137: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  inputs = _infer_schema(model_input)
/home/codespace/.local/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
